In [ ]:
import pandas as pd
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from tqdm import tqdm
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
# Chrome 설정
def ChromeSetup():
    options = webdriver.ChromeOptions()
    options.add_argument("--ignore-ssl-errors=yes")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-images")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-popup-blocking")
    return options


# 요소 찾기 재시도 함수
def retry_find_element(driver, locator, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            # dimmedLayer가 있으면 제거
            if driver.find_elements(By.ID, "dimmedLayer"):
                driver.execute_script("document.getElementById('dimmedLayer').style.display = 'none';")

            element = driver.find_element(*locator)
            return element
        except Exception as e:
            time.sleep(1)
            attempt += 1
    raise Exception(f"Could not find element after {retries} retries")

In [2]:
# 숙소 검색 및 처리 함수
def search_lodging(driver, lodging, error_lodgings):
    lodging_name = lodging["name"]
    lodging_sigungu = lodging["sigungu"]
    lodging_id = lodging["lodging_id"]

    searchloc = f"{lodging_sigungu} {lodging_name}"

    try:
        # 검색창을 찾아서 검색어 입력
        search_area = retry_find_element(driver, (By.ID, "search.keyword.query"))
        search_area.clear()  # 검색창 초기화
        search_area.send_keys(searchloc)

        # JavaScript로 검색 버튼 강제 클릭
        driver.execute_script("document.getElementById('search.keyword.submit').click()")

        # 검색 결과 기다림 (재시도 로직 추가)
        attempt = 0
        while attempt < 3:
            try:
                element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "moreview")))
                break  # 요소를 성공적으로 찾으면 루프 탈출
            except StaleElementReferenceException:
                attempt += 1
                time.sleep(1)

        if attempt == 3:
            raise Exception("Failed to find the element after 3 retries due to stale element reference")

        # a 태그에서 href 속성 추출
        href_value = element.get_attribute("href")

        # href에서 장소 번호 추출
        match = re.search(r"com/(\d+)", href_value)
        if match:
            place_number = match.group(1)
            return {
                "lodging_id": lodging_id,
                "name": lodging_name,
                "sigungu": lodging_sigungu,
                "place_number": place_number,
                "href": href_value,
            }
        else:
            return None

    except Exception as e:
        error_lodgings.append(
            {"lodging_id": lodging_id, "name": lodging_name, "sigungu": lodging_sigungu, "error_message": str(e)}
        )
        return None


# MySQL 데이터베이스 연결
connection = pymysql.connect(
    host="capstone-db.cfgmik22w69x.ap-northeast-2.rds.amazonaws.com",
    user="root",
    password="daejin2019",
    db="capstone",
    port=3306,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor,
)

# MySQL로부터 숙소 데이터 가져오기
sql = "SELECT lodging_id, name, sigungu FROM lodgings"
with connection.cursor() as cursor:
    cursor.execute(sql)
    lodgings = cursor.fetchall()

connection.close()

In [ ]:


# Chrome 드라이버를 4개 초기화하여 순차적으로 재사용
options = ChromeSetup()
drivers = [webdriver.Chrome(options=options) for _ in range(4)]

# 모든 드라이버에서 Kakao Maps의 시작 페이지로 이동 (한번만 이동)
search_url = "https://map.kakao.com"
for driver in drivers:
    driver.get(search_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "search.keyword.query")))


# 4개의 드라이버를 순환하며 검색어 변경
def process_lodgings(lodgings, drivers, error_lodgings):
    data = []  # 검색 결과를 저장할 리스트

    # tqdm progress bar 설정
    with tqdm(total=len(lodgings)) as progress_bar:
        for i, lodging in enumerate(lodgings):
            driver = drivers[i % len(drivers)]  # 4개의 드라이버를 순차적으로 사용
            result = search_lodging(driver, lodging, error_lodgings)
            if result is not None:  # 성공한 항목만 추가
                data.append(result)
            progress_bar.update(1)  # 진행 바 업데이트

    return data

In [3]:
# 최대 3번 재시도하는 로직
def retry_failed_lodgings(error_lodgings, drivers, retries=3):
    for attempt in range(retries):
        if not error_lodgings:
            break  # 더 이상 실패한 항목이 없으면 중단
        retry_data = []
        retry_data = process_lodgings(error_lodgings, drivers, [])

        # 성공적으로 재시도된 항목만 추가하고 실패 항목 갱신
        error_lodgings = [item for item in error_lodgings if item not in retry_data]
    return retry_data

In [4]:
# 1차 숙소 검색 진행
error_lodgings = []  # 실패한 항목 저장
data = process_lodgings(lodgings, drivers, error_lodgings)

# 실패한 항목 재시도
if error_lodgings:
    retry_data = retry_failed_lodgings(error_lodgings, drivers, retries=3)
    data.extend(retry_data)  # 재시도 성공한 항목을 data에 추가

# 모든 드라이버 종료
for driver in drivers:
    driver.quit()

  0%|          | 0/2991 [00:00<?, ?it/s]

100%|██████████| 397/397 [07:26<00:00,  1.12s/it]


In [13]:
# 결과를 DataFrame으로 변환 및 저장
df = pd.DataFrame(data)
df2 = df.drop_duplicates()

In [14]:
print(len(df), len(df2))

2922 2922


In [6]:

df.to_csv("../data/lodging_place_number(kakao).csv", index=False, encoding='utf-8-sig')

# 최종 실패 목록 출력
print("Final error lodgings:", error_lodgings)
error_lodgings.to_csv("../data/lodging_error.csv", index=False, encoding='utf-8-sig')

NameError: name 'df' is not defined

In [4]:

a = pd.read_csv('../data/lodging_place_number(kakao).csv')
a

,lodging_id,name,sigungu,place_number,href
0,1,가경재,안동시,2032317295,https://place.map.kakao.com/2032317295
1,2,가락관광호텔,송파구,10955780,https://place.map.kakao.com/10955780
2,3,가락청,전주시,1627968097,https://place.map.kakao.com/1627968097
3,4,가람나무,파주시,20886386,https://place.map.kakao.com/20886386
4,6,가름게스트하우스,서귀포시,14572405,https://place.map.kakao.com/14572405
...,...,...,...,...,...
2917,3766,KT&G 상상마당 부산 스테이,부산진구,418294986,https://place.map.kakao.com/418294986
2918,3778,mujuresortel,무주군,230165031,https://place.map.kakao.com/230165031
2919,3810,Valley모텔,경주시,25037735,https://place.map.kakao.com/25037735
2920,3813,V모텔,통영시,1817160004,https://place.map.kakao.com/1817160004


In [5]:
# lodgings를 데이터프레임으로 변환
lodgings_df = pd.DataFrame(lodgings)

# a에는 없고 lodgings에는 있는 lodging_id 찾기
missing_lodging_ids = lodgings_df[~lodgings_df['lodging_id'].isin(a['lodging_id'])]

# missing_lodging_ids를 b에 저장
b = missing_lodging_ids
b

,lodging_id,name,sigungu
17,19,가족사랑바다체험펜션,남해군
73,97,경주 노벰버키즈앤스파,경주시
129,167,골든튤립 호텔 남강,진주시
130,169,골든튤립인천에어포트호텔,중구
155,200,구름아래펜션,평창군
...,...,...,...
2561,3238,한탄강 디퍼펜션,철원군
2717,3428,호텔M-Tower,김포시
2787,3527,호텔인,정선군
2889,3674,1박2일 남해펜션,남해군


In [3]:
error_lodgings = []


In [16]:
# 검색 수행
processed_data = process_lodgings(b.to_dict('records'), drivers, error_lodgings)


100%|██████████| 69/69 [03:27<00:00,  3.01s/it]


In [17]:
processed_data

[]